In [87]:
from keras.layers import Dense,Conv2D, Flatten, MaxPooling2D, Dropout,Activation,BatchNormalization
from keras.models import Sequential,Model
from keras.optimizers import Adamax
import keras
import pandas as pd
import numpy as np
import os
import scipy
import warnings

warnings.filterwarnings('ignore')

## Loading the data

In [88]:
path_ = r"C:\Users\jbw738\ALL_python_notebooks\DL_tutorial\Transfer_Learning\fruits-360_dataset_2018_05_26\fruits-360"

In [89]:
training_fruit_img = []
training_label = []
for folder in os.listdir(path_+"\Training") :
    for img in os.listdir(path_+"\Training"+"\\"+folder) :
        temp = scipy.misc.imread(path_+"\Training"+"\\"+folder+"\\"+img)
        temp = scipy.misc.imresize(temp,(64,64))
        training_fruit_img.append(temp)
        training_label.append(folder)

In [90]:
training_fruit_img = np.array(training_fruit_img)
training_label = np.array(training_label)

In [91]:
label_to_id = {k:v for v,k in enumerate(np.unique(training_label))}
id_to_label = {v:k for v,k in enumerate(np.unique(training_label))}

In [92]:
training_label_id = np.array([label_to_id[i] for i in training_label])

In [93]:
training_fruit_img.shape,training_label_id.shape

((31688, 64, 64, 3), (31688,))

In [94]:
validation_fruit_img = []
validation_label = []
for folder in os.listdir(path_+"\Validation") :
    for img in os.listdir(path_+"\Validation"+"\\"+folder) :
        temp = scipy.misc.imread(path_+"\Validation"+"\\"+folder+"\\"+img)
        temp = scipy.misc.imresize(temp,(64,64))
        validation_fruit_img.append(temp)
        validation_label.append(folder)

validation_fruit_img = np.array(validation_fruit_img)
validation_label = np.array(validation_label)    

validation_label_id = np.array([label_to_id[i] for i in validation_label])

In [95]:
validation_fruit_img.shape,validation_label.shape

((10657, 64, 64, 3), (10657,))

In [96]:
x_train, x_test = training_fruit_img, validation_fruit_img
y_train, y_test = training_label_id, validation_label_id

In [97]:
x_train = x_train/255
x_test = x_test/255

In [98]:
x_flat_train = x_train.reshape(x_train.shape[0],64*64*3)
x_flat_test = x_test.reshape(x_test.shape[0],64*64*3)

In [99]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

## Model

In [100]:
num_classes = len(np.unique(training_label))

In [105]:
model = Sequential()

In [106]:
model.add(Conv2D(32,kernel_size=(3,3),input_shape=(64,64,3),padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,kernel_size=(3,3),padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization_7 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 32)        0         
__________

In [107]:
model.compile(optimizer=Adamax(),
              loss='categorical_crossentropy',
             metrics = ["accuracy"])

In [108]:
history = model.fit(x=x_train,
         y=y_train,
         batch_size = 64,
         epochs = 5,
         verbose=1,
         validation_data = (x_test,y_test))

Train on 31688 samples, validate on 10657 samples
Epoch 1/5
31688/31688 [==============================] - 566s 18ms/step - loss: 0.5328 - acc: 0.8818 - val_loss: 0.3465 - val_acc: 0.9137
Epoch 2/5
31688/31688 [==============================] - 584s 18ms/step - loss: 0.0501 - acc: 0.9889 - val_loss: 0.2179 - val_acc: 0.9397
Epoch 3/5
31688/31688 [==============================] - 583s 18ms/step - loss: 0.0343 - acc: 0.9904 - val_loss: 0.2806 - val_acc: 0.9249
Epoch 4/5
31688/31688 [==============================] - 585s 18ms/step - loss: 0.0304 - acc: 0.9909 - val_loss: 0.4537 - val_acc: 0.9016
Epoch 5/5
31688/31688 [==============================] - 583s 18ms/step - loss: 0.0263 - acc: 0.9910 - val_loss: 0.2535 - val_acc: 0.9301
